COMP 472 - Mini Project 1 - Fall 2021
Due October 18th, 2021
Team Samurai: Adrien Kamran (40095393), Abdul Sirawan (?)

**TASK 2**

**STEP 2 - Load the dataset in Python.**

In [ ]:
import pandas
dataset = pandas.read_csv("drug200.csv")
# Print the first 5 rows (header incl.) to see if this works as intended
print(dataset.head(5))

**STEP 3 - Plot the distribution of the instances in each class, then store the graphic in a file called "drug-distribution.pdf".**

In [ ]:
import matplotlib.pyplot as plt
# For each column in the dataset...
for column in dataset:
    # Create a histogram of the data in the column's rows
    plt.hist(dataset[column])
    plt.title = column
    # Save the figure as 'column_name.png'
    plt.savefig(fname=column)
    # Clear the plot's figure window to make room for the next one
    plt.clf()

# Assemble the PDF in an image editor to make the collage, save as PDF

Preliminary observations:
- Age: largest group between 40 and 50 yrs, smallest between 50 and 55 yrs.
- Sex: nearly even split.
- BP: more high than low, more low than normal.
- Cholesterol: more high than normal.
- Na to K: largest group between 10 and 15, smallest between 38 and 38.
- Drug: more drug Y than anything else, less drug C than anything else.

**STEP 4 - For the dataset, convert ordinal features to ordered numerical, and nominal features to unordered numerical.**

In [ ]:
# from pandas.api.types import CategoricalDtype

# Convert unordered nominal features
dataset = pandas.get_dummies(dataset, columns=['Sex'], prefix=['Sex'])
# Convert ordered nominal features to their numeric codes
dataset['BP'] = dataset['BP'].apply(lambda x: ['LOW', 'NORMAL', 'HIGH'].index(x))
dataset['Cholesterol'] = dataset['Cholesterol'].apply(lambda x: ['NORMAL', 'HIGH'].index(x))
# Convert target classes to arbitrary numeric codes
dataset['Drug'].replace(to_replace=['drugA', 'drugB', 'drugC', 'drugX', 'drugY'], value=[1, 2, 3, 4, 5], inplace=True)
# Swap columns due to get_dummies placing new columns at the end of the dataframe
cols = list(dataset.columns)
a, b = cols.index('Sex_M'), cols.index('Drug')
cols[a], cols[b] = cols[b], cols[a]
dataset = dataset[cols]
print(dataset.head(5))

# Create ordered categories for ordinal features
# bp_cat = CategoricalDtype(categories = ['LOW', 'NORMAL', 'HIGH'], ordered = True)
# chol_cat = CategoricalDtype(categories = ['NORMAL', 'HIGH'], ordered = True)
# Assign ordered categories to respective features
# dataset['BP'] = dataset['BP'].astype(bp_cat)
# dataset['Cholesterol'] = dataset['Cholesterol'].astype(chol_cat)
# Find columns with category data type
# cat_columns = dataset.select_dtypes(['category']).columns
# dataset[cat_columns] = dataset[cat_columns].apply(lambda x: x.cat.codes)
# drug_cat = CategoricalDtype(categories = ['drugA', 'drugB', 'drugC', 'drugX', 'drugY'], ordered = False)
# dataset['Drug'] = dataset['Drug'].astype(drug_cat)
# Create unordered categories for nominal features
# sex_cat = CategoricalDtype(categories = ['M', 'F'], ordered = False)
# Assign unordered categories to respective features
# dataset['Sex'] = dataset['Sex'].astype(sex_cat)

**STEP 5 - Split the dataset**

In [ ]:
import sklearn.model_selection as skl
# First, split dataset into features and targets
features = dataset.iloc[:, 0:6] # The first five columns are features
target = dataset.iloc[:, 6:] # The last column is  the target
print(features.head(5))
print(target.head(5))
# Assign training and testing data to respective arrays
features_train, features_test, target_train, target_test = skl.train_test_split(features, target)
# Save training and testing data to files
features_train.to_csv(path_or_buf = 'features_train.csv')
features_test.to_csv(path_or_buf = 'features_test.csv')
target_train.to_csv(path_or_buf = 'target_train.csv')
target_test.to_csv(path_or_buf = 'target_test.csv')

**STEP 6 - Run classifiers**

**STEP 6.a - NB**

In [ ]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
# Create the Gaussian Classifier
gnb = GaussianNB()
# Train the model
gnb.fit(features_train, target_train.values.ravel())
target_prediction = gnb.predict(features_test)
# Display the results
print("a)\n*****************************************************************************\nGAUSSIAN NAIVE BAYES\n")
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction))
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None))
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"))

**STEP 6.b - Base-DT**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# Create the DT Classifier
dtc = DecisionTreeClassifier()
# Train the model
dtc.fit(features_train, target_train.values.ravel())
target_prediction = dtc.predict(features_test)
# Display the results
print("a)\n*****************************************************************************\nBASE DECISION TREE\n")
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction))
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None))
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"))

**STEP 6.c - Top-DT**

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np
# Establish hyper-parameters to be used in evaluation
tree_parameters = {
      'criterion':['entropy', 'gini'],
      'max_depth':np.arange(3, 10),
      'min_samples_split':[3, 4, 5]
}
# Create an instance of the GridSearchCV
gscv = GridSearchCV(DecisionTreeClassifier(), tree_parameters)
# Run the exhaustive grid search on hyper-parameters, given the test data
gscv.fit(features_train, target_train.values.ravel())
# Retrieve the best estimator (which uses the best parameters)
print(gscv.best_estimator_)
# Use the best estimator in the decision tree classifier
target_prediction = gscv.best_estimator_.predict(features_test)
# Display the results
print("a)\n*****************************************************************************\nTOP DECISION TREE\n"
      "Best hyper-parameters:", gscv.best_params_, "\n")
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction))
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None))
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"))

**STEP 6.d - PER**

In [ ]:
from sklearn.linear_model import Perceptron
# Create the Perceptron classifier
pc = Perceptron()
# Train the model
pc.fit(features_train, target_train.values.ravel())
target_prediction = pc.predict(features_test)
# Display the results
print("a)\n*****************************************************************************\nPERCEPTRON\n")
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction))
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None))
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"))

**STEP 6.e - Base-MLP**

In [ ]:
from sklearn.neural_network import MLPClassifier
# Create the MLP classifier with instructed parameters
mlpc = MLPClassifier(hidden_layer_sizes=100, activation="logistic", solver="sgd")
# Train the model
mlpc.fit(features_train, target_train.values.ravel())
target_prediction = mlpc.predict(features_test)
# Display the results
print("a)\n*****************************************************************************\nBASE MULTI-LAYERED PERCEPTRON\n")
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction))
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None))
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"))

**STEP 6.e - Top-MLP**

In [ ]:
# Establish hyper-parameters to be used in evaluation
mlpc_parameters = {
      'activation':['identity', 'logistic', 'tanh', 'relu'],
      'hidden_layer_sizes':[(30, 50), (10, 10, 10)],
      'solver':['adam', 'sgd']
}
# Create an instance of the GridSearchCV
gscv = GridSearchCV(MLPClassifier(), mlpc_parameters)
# Run the exhaustive grid search on hyper-parameters, given the test data
gscv.fit(features_train, target_train.values.ravel())
# Retrieve the best estimator (which uses the best parameters)
print(gscv.best_estimator_)
# Use the best estimator in the decision tree classifier
target_prediction = gscv.best_estimator_.predict(features_test)
# Display the results
print("a)\n*****************************************************************************\nTOP MULTI-LAYERED PERCEPTRON\n"
      "Best hyper-parameters:", gscv.best_params_, "\n")
print("b)\nConfusion Matrix:\n", metrics.confusion_matrix(target_test, target_prediction))
print("c)\nPrecision:\n", metrics.confusion_matrix(target_test, target_prediction),
      "\nRecall:\n", metrics.recall_score(target_test, target_prediction, average=None),
      "\nF1-measure:\n", metrics.f1_score(target_test, target_prediction, average=None))
print("d)\nAccuracy:\n", metrics.accuracy_score(target_test, target_prediction),
      "\nMacro-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="macro"),
      "\nWeighted-average F1-measure:\n", metrics.f1_score(target_test, target_prediction, average="weighted"))